# Cálculo Solar Fotovoltaico - Versão Simples

Notebook criado do zero com abordagem direta e funcional.

In [80]:
!pip install pvlib pandas numpy requests matplotlib seaborn

In [9]:
# Instalação e imports
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from datetime import datetime


# PVLib
import pvlib
from pvlib import location, pvsystem, modelchain, irradiance, solarposition
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS


print(f"pvlib versão: {pvlib.__version__}")
print("✅ Imports OK")

pvlib versão: 0.13.0
✅ Imports OK


In [78]:
potencia_modulo = 605  # W
num_modulos = 300       # Número de módulos

lat = -23.7619
lon = -53.3290

inclinacao = 90
orientacao = 180

modelo_transposicao = 'perez'  # Modelo de transposição

losses_parameters = {
        'soiling': 2.0,
        'shading': 3,
        'mismatch': 2.5,
        'wiring': 2.0
    }

module_parameters = {
    # Coeficientes de temperatura (do datasheet)
    'alpha_sc': 0.00046,      # +0.046%/°C para Isc
    'beta_oc': -0.0025,       # -0.25%/°C para Voc  
    'gamma_r': -0.0029,       # -0.29%/°C para Pmax
    
    # Parâmetros do modelo de diodo único
    'a_ref': 1.8,             # Fator de idealidade (típico para TOPCon)
    'I_L_ref': 14.86,         # Corrente fotovoltaica em STC (baseado no Isc médio)
    'I_o_ref': 2.5e-12,       # Corrente de saturação (estimado para TOPCon)
    'R_s': 0.25,              # Resistência série (estimado para alta eficiência)
    'R_sh_ref': 450.0,        # Resistência shunt (típico para módulos premium)
    
    # Configuração física
    'cells_in_series': 132,    # 132 células (6x22 conforme datasheet)
    
    # Parâmetros STC (do datasheet - valores médios)
    'STC': potencia_modulo,               # Potência nominal em Watts
    'V_oc_ref': 48.0,         # Tensão circuito aberto
    'I_sc_ref': 14.86,        # Corrente curto circuito  
    'V_mp_ref': 45.0,         # Tensão máxima potência
    'I_mp_ref': 13.44,        # Corrente máxima potência (605W/45V)
    
    # Coeficientes para modelo de temperatura Sandia
    'A0': -3.47,              # Ajustado para TOPCon
    'A1': -0.0594,
    'A2': 0.0,
    'A3': 0.0,
    'A4': 0.0,
    'B0': 0.0,
    'B1': 0.0,
    'B2': 0.0,
    'B3': 0.0,
    'B4': 0.0,
    'B5': 0.0,
    'DTC': 3.0                # Delta temperatura célula
}

inverter_parameters = {
    'Paco': 100000,
    'Pdco': 150000,
    'Vdco': 615,              # Tensão CC nominal (615V do datasheet)
    'Pso': 5.0,               # Autoconsumo noturno (<5W do datasheet)
    
    # Coeficientes de eficiência (ajustados para 98.11% máx. eficiência)
    'C0': -0.000005,          # Ajustado para alta eficiência
    'C1': -0.000100,          
    'C2': 0.001200,           
    'C3': -0.018000,          
    'Pnt': 0.02               # Potência de threshold normalizada
}

In [52]:
# Função PVGIS simplificada
def buscar_dados_pvgis(lat, lon, startyear=2005, endyear=2020):
    """Busca dados PVGIS de forma simples"""
    
    url = (f"https://re.jrc.ec.europa.eu/api/v5_2/seriescalc?"
           f"lat={lat}&lon={lon}&"
           f"startyear={startyear}&endyear={endyear}&"
           f"outputformat=json&usehorizon=1&selectrad=1&angle=0&aspect=0")
    
    print(f"Buscando dados para {lat}, {lon} do PVGIS...")
    
    response = requests.get(url, timeout=60)
    data = response.json()
    
    # Extrair dados
    records = []
    for record in data['outputs']['hourly']:
        time_str = record['time']
        dt = pd.to_datetime(time_str, format='%Y%m%d:%H%M', utc=True)
        
        records.append({
            'datetime': dt,
            'ghi': float(record['G(i)']),
            'temp_air': float(record.get('T2m', 25)),
            'wind_speed': float(record.get('WS10m', 2))
        })
    
    df = pd.DataFrame(records)
    df.set_index('datetime', inplace=True)
    df.index = df.index.tz_convert('America/Sao_Paulo')
    
    print(f"✅ {len(df)} registros | GHI: {df['ghi'].min():.0f}-{df['ghi'].max():.0f} W/m²")
    return df

# Teste
df_teste = buscar_dados_pvgis(lat, lon)
print(f"Registros com GHI > 0: {(df_teste['ghi'] > 0).sum()}")

Buscando dados para -23.7619, -53.329 do PVGIS...
✅ 140256 registros | GHI: 0-1147 W/m²
Registros com GHI > 0: 67549


In [53]:
# Função cálculo completa
def calcular_sistema_solar(lat, lon, inclinacao, azimute, num_modulos, losses_parameters, module_parameters, inverter_parameters, potencia_modulo=540, modelo_transposicao='perez'):
    """Calcula geração solar de forma direta"""
    
    print(f"🔄 Sistema: {num_modulos}x{potencia_modulo}W = {num_modulos*potencia_modulo/1000:.1f}kWp")
    
    # 1. Dados meteorológicos
    df = buscar_dados_pvgis(lat, lon)
    
    # 2. Localização
    site = location.Location(latitude=lat, longitude=lon)
    
    # 3. Posição solar
    solar_pos = solarposition.get_solarposition(df.index, lat, lon)
    
    # 4. Decomposição irradiação
    decomp = irradiance.louche(ghi=df['ghi'], solar_zenith=solar_pos['zenith'], datetime_or_doy=df.index)
    
    # 5. Preparar dados para ModelChain
    weather = pd.DataFrame({
        'ghi': df['ghi'],
        'dni': decomp['dni'],
        'dhi': decomp['dhi'],
        'temp_air': df['temp_air'],
        'wind_speed': df['wind_speed']
    }, index=df.index)
    
    print(f"✅ Dados: GHI={weather['ghi'].mean():.0f}, DNI={weather['dni'].mean():.0f}, DHI={weather['dhi'].mean():.0f}")

    
    system = pvsystem.PVSystem(
        surface_tilt=inclinacao,
        surface_azimuth=azimute,
        module_parameters=module_parameters,
        inverter_parameters=inverter_parameters,
        modules_per_string=num_modulos,
        strings_per_inverter=1,
        temperature_model_parameters=TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass'],
        losses_parameters=losses_parameters
    )
    
    # 7. ModelChain
    mc = modelchain.ModelChain(
        system,
        site,
        transposition_model=modelo_transposicao
    )
    
    # 8. Simulação
    print("🔄 Executando simulação...")
    mc.run_model(weather)
    
    # 9. Resultados
    if mc.results.ac is None:
        print("❌ Erro: Sem resultados AC")
        return None
    
    ac_power = mc.results.ac.fillna(0).clip(lower=0)
    annual_energy = ac_power.sum() / 1000 / 16  # kWh, 16 anos de dados
    
    # Estatísticas
    potencia_total = num_modulos * potencia_modulo / 1000
    yield_especifico = annual_energy / potencia_total
    fator_capacidade = (annual_energy / (potencia_total * 8760)) * 100
    
    # Mensal
    monthly = ac_power.resample('M').sum() / 1000
    monthly_avg = monthly.groupby(monthly.index.month).mean()
    
    print(f"✅ SUCESSO!")
    print(f"⚡ Geração anual: {annual_energy:.0f} kWh")
    print(f"🎯 Yield específico: {yield_especifico:.0f} kWh/kWp")
    print(f"📊 Fator capacidade: {fator_capacidade:.1f}%")
    
    return {
        'annual_energy_kwh': annual_energy,
        'yield_especifico': yield_especifico,
        'fator_capacidade': fator_capacidade,
        'monthly_avg': monthly_avg.to_dict(),
        'ac_power': ac_power,
        'weather': weather
    }

print("✅ Função definida!")

✅ Função definida!


In [79]:
# Executar cálculo
resultado = calcular_sistema_solar(
    lat=lat,
    lon=lon, 
    inclinacao=inclinacao,
    azimute=orientacao,
    num_modulos=num_modulos,
    losses_parameters=losses_parameters,
    modelo_transposicao=modelo_transposicao,
    potencia_modulo=potencia_modulo,
    module_parameters=module_parameters,
    inverter_parameters=inverter_parameters
)

if resultado:
    print("\n🎯 === RESULTADOS FINAIS ===")
    print(f"⚡ Geração anual: {resultado['annual_energy_kwh']:.0f} kWh")
    print(f"🎯 Yield específico: {resultado['yield_especifico']:.0f} kWh/kWp")
    print(f"📊 Fator capacidade: {resultado['fator_capacidade']:.1f}%")
    
    print("\n📅 Geração Mensal:")
    meses = ['Jan','Fev','Mar','Abr','Mai','Jun','Jul','Ago','Set','Out','Nov','Dez']
    for i, mes in enumerate(meses, 1):
        valor = resultado['monthly_avg'].get(i, 0)
        print(f"  {mes}: {valor:.0f} kWh")

🔄 Sistema: 300x605W = 181.5kWp
Buscando dados para -23.7619, -53.329 do PVGIS...
✅ 140256 registros | GHI: 0-1147 W/m²
✅ Dados: GHI=209, DNI=206, DHI=68
🔄 Executando simulação...
✅ SUCESSO!
⚡ Geração anual: 386196 kWh
🎯 Yield específico: 2128 kWh/kWp
📊 Fator capacidade: 24.3%

🎯 === RESULTADOS FINAIS ===
⚡ Geração anual: 386196 kWh
🎯 Yield específico: 2128 kWh/kWp
📊 Fator capacidade: 24.3%

📅 Geração Mensal:
  Jan: 35498 kWh
  Fev: 32094 kWh
  Mar: 34186 kWh
  Abr: 29852 kWh
  Mai: 29265 kWh
  Jun: 28399 kWh
  Jul: 29633 kWh
  Ago: 30115 kWh
  Set: 31258 kWh
  Out: 33500 kWh
  Nov: 35553 kWh
  Dez: 34675 kWh


/var/folders/2b/5vjdl32570s6mf5lnfl1clph0000gn/T/ipykernel_42636/3945529327.py:67: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = ac_power.resample('M').sum() / 1000
